In [134]:
import numpy as np
from numpy.linalg import eigh, eig
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
from scipy.optimize import curve_fit

import importlib

import PCA
importlib.reload(PCA)


from jinja2 import Template

output=r"../slides/index-A.html"
input = r"../slides/index-to-build.html"

In [139]:
roi = np.array([[1400,1440]])
map = PCA.Map('../data/GA29/mapa3_laser532nm_100_mocy_1akumulacja_1s_CRR.txt', begin=1250, end=1450, baseline='arPLS', roi=roi)

C:\Users\PJ\Documents\Studia\Analiza\.venv\Lib\site-packages\rampy\baseline.py:240: RuntimeWarning:

overflow encountered in exp

C:\Users\PJ\Documents\Studia\Analiza\.venv\Lib\site-packages\rampy\baseline.py:240: RuntimeWarning:

overflow encountered in exp

C:\Users\PJ\Documents\Studia\Analiza\.venv\Lib\site-packages\rampy\baseline.py:240: RuntimeWarning:

overflow encountered in exp

C:\Users\PJ\Documents\Studia\Analiza\.venv\Lib\site-packages\rampy\baseline.py:240: RuntimeWarning:

overflow encountered in exp

C:\Users\PJ\Documents\Studia\Analiza\.venv\Lib\site-packages\rampy\baseline.py:240: RuntimeWarning:

overflow encountered in exp

C:\Users\PJ\Documents\Studia\Analiza\.venv\Lib\site-packages\rampy\baseline.py:240: RuntimeWarning:

overflow encountered in exp

C:\Users\PJ\Documents\Studia\Analiza\.venv\Lib\site-packages\rampy\baseline.py:240: RuntimeWarning:

overflow encountered in exp

C:\Users\PJ\Documents\Studia\Analiza\.venv\Lib\site-packages\rampy\baseline.py:240: Runtim

In [140]:
map.plotSpectrums(0, 10, 30, 100, 200, 1000, 1500).show()

In [138]:
import rampy
roi = np.array([[1400,1440]])
spec = 1

ycalc, baseline = rampy.baseline(map.raman_shift, map.data_matrix[spec], roi, 'arPLS')
# plot spectrum and baseline

fig = go.Figure()
fig.add_scatter(x=map.raman_shift, y=map.data_matrix[spec], mode='lines', name='Spectrum')
fig.add_scatter(x=map.raman_shift, y=baseline.T[0], mode='lines', name='Baseline')
fig.show()

C:\Users\PJ\Documents\Studia\Analiza\.venv\Lib\site-packages\rampy\baseline.py:240: RuntimeWarning:

overflow encountered in exp



In [ ]:
map.plotWeights(False, False).show()

In [141]:
map.plotBaseVectors(4).show()

In [100]:
map.refit_vectors = 4

map.refit_matrices = []
map.addRefitMatrix(np.array([
    [1, 1, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
]), True)

map.addRefitMatrix(np.array([
    [1, 0, 0, 0],
    [-0.17, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
]), True)

map.addRefitMatrix(np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0.4, 0, 1, -7],
    [0, 0, 0, 1],
]), True)

map.addRefitMatrix(np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
]), True)

map.plotBaseVector(0, True, True).show()
map.plotComponent(0, True).show()
map.plotBaseVector(1, True, True).show()
map.plotComponent(1, True).show()
map.plotBaseVector(2, True, True).show()
map.plotComponent(2, True).show()
map.plotBaseVector(3, True, True).show()
map.plotComponent(3, True).show()

# map.plotComponent(0, True).show()

In [ ]:
# For abstract

# [] Trzeba zrobić trochę większe napisy na osiach (większe i może trochę grubsze). 
# [X] Zamiast "10 BN" i "11 BN" lepiej dać "h10BN" i "epitaxial hBN" (ten drugi nie jest izotopowo czysty z 11B tylko jest to naturalny BN z mieszanką 10B i 11B). 
# [X] Poza tym w podpisie oznaczasz kolejne panele jako (a), (b), (c), a na samych panelach nie ma tych oznaczeń - trzeba je dodać.

import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar

tfs = 24
fs = 19
fig, axs = plt.subplots(1, 3, figsize=(17, 5))

# Plot for new_base_vectors[0]
axs[0].plot(map_a.raman_shift, new_base_vectors[0]/np.max(new_base_vectors[0]), label='h$^{10}$BN')
axs[0].plot(map_a.raman_shift, new_base_vectors[2]/np.max(new_base_vectors[0]), label='epitaxial\nhBN')
axs[0].set_title('(a) PCA obtained spectra', fontsize=tfs, fontweight='bold')
axs[0].set_ylabel('Intensity [ab. units]', fontsize=fs)
axs[0].set_xlabel(r'Raman shift [$\text{cm}^{-1}$]', fontsize=fs)
axs[0].legend(fontsize=18, loc='upper left')

tmp = np.reshape(new_vectors[0], map_a.shape)
tmp[tmp < 0] = 0
tmp/=np.max(tmp)
im = axs[1].imshow(tmp, extent=[0,10,0,10], aspect=1)
# cax = fig.add_axes([axs[1].get_position().x1+0.01,axs[1].get_position().y0,0.02,axs[1].get_position().height])
cbar = fig.colorbar(im, ax=axs[1])#, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Intensity [ab. units]', fontsize=fs)
cbar.ax.tick_params(labelsize=16)
axs[1].set_title('(b) h$\mathbf{^{10}}$BN', fontsize=tfs, fontweight='bold')
axs[1].set_xlabel('x [$\mu$m]', fontsize=fs)
axs[1].set_ylabel('y [$\mu$m]', fontsize=fs)

tmp = np.reshape(new_vectors[2], map_a.shape)
tmp[tmp < 0] = 0
tmp/=np.max(tmp)
im = axs[2].imshow(tmp, extent=[0,10,0,10], aspect=1)
# cax = fig.add_axes([axs[2].get_position().x1+0.01,axs[2].get_position().y0,0.02,axs[2].get_position().height])
cbar = fig.colorbar(im, ax=axs[2])#, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Intensity [ab. units]', fontsize=fs)
cbar.ax.tick_params(labelsize=16)
axs[2].set_title(r'(c) epitaxial hBN', fontsize=tfs, fontweight='bold')
axs[2].set_xlabel('x [$\mu$m]', fontsize=fs)
axs[2].set_ylabel('y [$\mu$m]', fontsize=fs)

for i in range(3):
    axs[i].tick_params(axis='both', which='major', labelsize=17)

plt.tight_layout()
plt.savefig('img/abstract.png')
plt.show()

# px.line(x = map_a.raman_shift, y = new_base_vectors[0])
# tmp = np.reshape(new_vectors[0], map_a.shape)
# tmp[tmp < 0] = 0
# px.imshow(tmp).show()
# 
# px.line(x = map_a.raman_shift, y = new_base_vectors[1]).show()
# tmp = np.reshape(new_vectors[1], map_a.shape)
# tmp[tmp < 0] = 0
# px.imshow(tmp).show()
# 
# px.line(x = map_a.raman_shift, y = new_base_vectors[2]).show()
# tmp = np.reshape(new_vectors[2], map_a.shape)
# tmp[tmp < 0] = 0
# px.imshow(tmp).show()


In [ ]:
plotly_jinja_data = {}
for i in range(7):
    plotly_jinja_data["a_base_" + str(i)] = px.line(x = map_a.raman_shift, y = new_base_vectors[i]).update_layout(
    autosize=False,
    width=700,
    height=500).to_html(full_html=False)
    plotly_jinja_data["a_map_" + str(i)] = px.imshow(np.reshape(new_vectors[i], map_a.shape)).update_layout(
    autosize=False,
    width=500,
    height=500).to_html(full_html=False)

for i in range(7):
    plotly_jinja_data["b_base_" + str(i)] = "{{b_base_" + str(i) + "}}"
    plotly_jinja_data["b_map_" + str(i)] = "{{b_map_" + str(i) + "}}"

for i in range(8):
    plotly_jinja_data["c_base_" + str(i)] = "{{c_base_" + str(i) + "}}"
    plotly_jinja_data["c_map_" + str(i)] = "{{c_map_" + str(i) + "}}"

with open(output, "w", encoding="utf-8") as output_file:
    with open(input) as template_file:
        j2_template = Template(template_file.read())
        output_file.write(j2_template.render(plotly_jinja_data))


In [ ]:
map_a.plotSpectrum(map_a.posToId(0, 0))

In [ ]:
px.scatter(x = new_vectors[3], y = new_vectors[4]).show()
px.scatter(x = new_vectors[4], y = new_vectors[5]).show()
px.scatter(x = new_vectors[3], y = new_vectors[5]).show()

In [ ]:
np.dot(map_a.vectors.T, map_a.vectors)

In [ ]:
map_a.plotBaseVectors(5)
r = np.dot(new_vectors.T, new_base_vectors) 
# d = map_a.data_matrix - r
i = map_a.posToId(5, 30)
f = map_a.plotSpectrum(i)
f.add_scatter(x=map_a.raman_shift, y=r[i], mode='lines', name='Rekonstrukcja')
f.add_scatter(x=map_a.raman_shift, y=new_vectors[0][i] * new_base_vectors[0], mode='lines')
f.add_scatter(x=map_a.raman_shift, y=new_vectors[1][i] * new_base_vectors[1], mode='lines')
f.add_scatter(x=map_a.raman_shift, y=new_vectors[2][i] * new_base_vectors[2], mode='lines')
f.add_scatter(x=map_a.raman_shift, y=new_vectors[3][i] * new_base_vectors[3], mode='lines')

              
# f.add_scatter(x=map_a.raman_shift, y=d[i], mode='lines', name='Roznica')
f.update_layout(hovermode="x")
f.show()